In [1]:
########################################################################
########################################################################
#                   SOCKETS
########################################################################
########################################################################

import cv2
import numpy as np
import numpy.lib.recfunctions as nlr
from struct import *


def reshapeBuffer(data, width, height):
    img = data.reshape((height, width, 3))
#     img = img * 1.0/255.0
    img = np.flip(img) # Convert from BGR to RGB
    img = np.flip(img, axis=1)
#     img = np.transpose(img, (1, 0, 2))
    # print(img.shape)
    return img


def send_msg(sock, msg):
    # Prefix each message with a 4-byte length (network byte order)
    msg = pack('I', len(msg)) + msg
    sock.send(msg)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data


In [2]:
########################################################################
########################################################################
#                   YOLO
########################################################################
########################################################################

import torch
import numpy as np
import cv2
from time import time

from PIL import Image
from PIL import ImageTk


class ObjectDetection:
    def __init__(self):
        self.model = self.load_model()
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("\n\nDevice Used:",self.device)

    def load_model(self):
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        #frame = [frame]
        results = self.model(frame)
     
        labels, cord = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]
        return labels, cord


    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]


    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        print("Predictions")
        print(n)
        for i in range(n):
            print(self.class_to_label(labels[i]))
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame


    def inference(self, img_in):
        start_time = time()
        results = self.score_frame(img_in)
        frame = self.plot_boxes(results, img_in) # Problem potentially lies here
        
        # Could be returning wrong frame because of scope of variable
        
        
        end_time = time()
        fps = 1 / np.round(end_time - start_time, 3)
        #print(f"Frames Per Second : {fps}")
        cv2.putText(frame, f'YOLO-FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
#         cv2.imshow("img", frame)
#         cv2.waitKey(0)
        return frame



########################################################################
########################################################################
#                   YOLO USAGE
########################################################################
########################################################################

# detection = ObjectDetection()

# #img = cv2.imread('C:/Brandon/CurrentProjects/CMUD/instant-ngp/data/nerf/DeskM/images/DESK-10.jpg')
# #img = cv2.imread('/home/bwilab/Documents/kynoch/CMUD/cmud-instant-ngp/data/nerf/AHG/AllAngles/export1920/images/IMG_9483.jpeg')
# img = cv2.imread('/home/bwilab/Documents/kynoch/CMUD/cmud-instant-ngp/data/nerf/DeskL/images/DESK-10.jpg')

# img_copy = img.copy()
# img_out = detection.inference(img_copy)

# cv2.imshow("Detection", img_out)
# cv2.waitKey(0)




In [3]:
import threading
import socket
from struct import *

########################################################################
########################################################################
#                   MAIN
########################################################################
########################################################################


def writeBufferToDisk(buffer):
    file = open("buffer", "wb")
    file.write(buffer)
    file.close()
    
def readBufferFromDisk():
    buffer_file = open("buffer", "rb")
    buffer = buffer_file.read()
    buffer_file.close()
    return buffer





ready_for_frame = threading.Event()
ready_for_frame.set()

img_ready = threading.Event()
img_ready.clear()

img_out = np.zeros((1920, 1080, 3), dtype=np.uint8)



def cvThread():
    cv2.namedWindow('Framedump') 
    cv2.startWindowThread()
    
    while (True):
        img_ready.wait()
        img_ready.clear()
        
        cv2.imshow('Framedump', img_out)
        for i in range (1,32):
#             print(i)
            cv2.waitKey(1)
        
        ready_for_frame.set()
        print("Set")




HOST = "127.0.0.1" #Localhost / loopback
PORT = 28501

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))

yolo_detection = ObjectDetection()

cv_thread = threading.Thread(target=cvThread, args=())
cv_thread.start()

### NERF LOOP #################################################
while (True):
    # Ping loop so that NeRF program continues while python is displaying frame
    # for long period of time
    while (not ready_for_frame.is_set()): ## TODO: THERE IS SOMETHING WRONG WITH THE SYNCHRONIZATION IN PYTHON HERE
        s.recv(1)
        s.send(b'\x00')
    
    s.recv(1)
    s.send(b'\x01')

    # Fetch and decode package header
    package_header = s.recv(8)
    frame_dimension = unpack('II', package_header)
    print(frame_dimension)
    width = frame_dimension[0]
    height = frame_dimension[1]

    # Fetch payload
    buffer = recvall(s, width * height * 3)
    
    print(len(buffer))
    send_msg(s, b'Received buffer') # Send response immediately to unblock thread in C++
    
#     writeBufferToDisk(buffer)
#     break


    np_buffer = np.frombuffer(buffer, dtype=np.uint8)
    img = reshapeBuffer(np_buffer, width, height)

    img_copy = img.copy()

    img_out = yolo_detection.inference(img_copy)
    
    ready_for_frame.clear()
    img_ready.set()

### NERF LOOP #################################################



### TESTING FROM BUFFER ###################################################
# yolo_detection = ObjectDetection()

# buffer = readBufferFromDisk()
# width = 1920
# height = 1080


# np_buffer = np.frombuffer(buffer, dtype=np.uint8)
# img = reshapeBuffer(np_buffer, width, height)

# img_copy = img.copy()

# img_out = yolo_detection.inference(img_copy)


# cv2.namedWindow('Framedump')
# cv2.startWindowThread()

# cv2.imshow('Framedump', img_out)
# cv2.waitKey(0)

    

# img = cv2.imread('/home/bwilab/Documents/kynoch/CMUD/cmud-instant-ngp/data/nerf/DeskL/framedump.png')

# img_copy = img.copy()
# img_out = yolo_detection.inference(img_copy)

# cv2.imshow("Detection", img_out)
# cv2.waitKey(0)
    
### TESTING FROM BUFFER ###################################################
    


Using cache found in /home/bwilab/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-9 Python-3.10.6 torch-1.13.0 CUDA:0 (NVIDIA GeForce RTX 3080, 10015MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
QObject::moveToThread: Current thread (0x84d6fe0) is not the object's thread (0x7fb6c4053a20).
Cannot move to target thread (0x84d6fe0)

QObject::moveToThread: Current thread (0x84d6fe0) is not the object's thread (0x7fb6c4053a20).
Cannot move to target thread (0x84d6fe0)

QObject::moveToThread: Current thread (0x84d6fe0) is not the object's thread (0x7fb6c4053a20).
Cannot move to target thread (0x84d6fe0)

QObject::moveToThread: Current thread (0x84d6fe0) is not the object's thread (0x7fb6c4053a20).
Cannot move to target thread (0x84d6fe0)

QObject::moveToThread: Current thread (0x84d6fe0) is not the object's thread (0x7fb6c4053a20).
Cannot move to target thread (0x84d6fe0)

QObject::moveToThread: Current thread (0x84d6fe0



Device Used: cuda
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
1
bench
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220

(1920, 1080)
6220800
Predictions
3
mouse
cup
keyboard
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
0
Set
(1920, 1080)
6220800
Predictions
1
mouse
Set
(1920, 1080)
6220800
Predictions
4
mouse
keyboard
book
tv
Set
(1920, 1080)
6220800
Predictions
3
keyboard
tv
mouse
Set
(1920, 1080)
6220800
Predictions
6
tv
keyboard
mouse
book
laptop
remote
Set
(1920, 1080)
6220800
Predictions
3
tv
mouse
keyboard
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
cell phone
tv
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
book
tv
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
tv
book
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
book
tv
Set
(1920, 1080)
6220800
Predictions
6
tv
mouse
keyboard
tv
book
cell phone
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
tv
book
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
tv
book
Set
(1920, 1080)
6220800
Predictions
5
tv
mouse
keyboard
tv
book
Set
(1920, 1080)
6220800
Predictio

(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse
keyboard
cell phone
tv
remote
book
Set
(1920, 1080)
6220800
Predictions
6
mouse

Set
(1920, 1080)
6220800
Predictions
5
cell phone
remote
tv
book
keyboard
Set
(1920, 1080)
6220800
Predictions
7
cell phone
remote
keyboard
tv
mouse
book
book
Set
(1920, 1080)
6220800
Predictions
4
mouse
keyboard
tv
book
Set
(1920, 1080)
6220800
Predictions
4
keyboard
mouse
tv
book
Set
(1920, 1080)
6220800
Predictions
3
keyboard
mouse
tv
Set
(1920, 1080)
6220800
Predictions
4
keyboard
mouse
tv
book
Set
(1920, 1080)
6220800
Predictions
3
keyboard
tv
mouse
Set
(1920, 1080)
6220800
Predictions
3
keyboard
mouse
tv
Set
(1920, 1080)
6220800
Predictions
3
keyboard
mouse
tv
Set
(1920, 1080)
6220800
Predictions
5
keyboard
mouse
mouse
tv
book
Set
(1920, 1080)
6220800
Predictions
4
keyboard
mouse
tv
book
Set
(1920, 1080)
6220800
Predictions
4
keyboard
tv
mouse
book
Set
(1920, 1080)
6220800
Predictions
5
keyboard
tv
book
mouse
book
Set
(1920, 1080)
6220800
Predictions
4
keyboard
tv
book
mouse
Set
(1920, 1080)
6220800
Predictions
4
cell phone
keyboard
book
mouse
Set
(1920, 1080)
6220800
Predictions

error: unpack requires a buffer of 8 bytes

In [ ]:
# test_img = np.zeros([500,500,3])

# test_img[:,:,2] = np.ones([500,500])*0/255.0 #R
# test_img[:,:,1] = np.ones([500,500])*0/255.0 #G
# test_img[:,:,0] = np.ones([500,500])*255/255.0 #B

# #cv2.imwrite('color_img.jpg', test_img)
# cv2.imshow("image", test_img)
# cv2.waitKey()